<a href="https://colab.research.google.com/github/viveka02/developer_tools_extension/blob/master/backend_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Part 1: Importing the Tools ---
# We're importing the necessary tools. FastAPI is our main framework.
# Pydantic helps us make sure the data we receive is in the right format.
# `os` and `httpx` are for talking to the external Claude AI.
from fastapi import FastAPI
from pydantic import BaseModel
import httpx
import os

# --- Part 2: Setting up our FastAPI "Kitchen" ---
# This line creates our main application. Think of it as opening the restaurant.
app = FastAPI()

# This is a secret key to talk to the Claude API.
# In a real app, we would load this securely.
# For this prototype, you can get one from Anthropic's website.
# NOTE: The variable name `ANTHROPIC_API_KEY` is just an example.
# We will use a secure way to manage this in the final product.
API_KEY = os.environ.get("ANTHROPIC_API_KEY", "YOUR_CLAUDE_API_KEY_HERE")
MODEL_NAME = "claude-3-opus-20240229"

# --- Part 3: Defining our "Order Slip" ---
# This class defines what an "order" from the frontend should look like.
# It MUST contain a "message" which is a string (text).
class UserMessage(BaseModel):
    message: str

# --- Part 4: The "Waiter" - Listening for Orders ---
# This is the main "endpoint". It's a specific URL our frontend will send messages to.
# When a POST request comes to "/chat", this function runs.
@app.post("/chat")
async def handle_chat(user_message: UserMessage):
    # This is where our magic happens. We take the user's message
    # and send it to our "specialist chef" (the Claude AI).
    ai_response = await call_claude_api(user_message.message)

    # We send the AI's response back to the frontend.
    return {"reply": ai_response}

# --- Part 5: Talking to the "Specialist Chef" (Claude) ---
# This is the most important function. It takes the user's message,
# combines it with our secret instructions, and gets a response.
async def call_claude_api(user_text: str):
    # THIS IS THE HEART OF OUR INTELLECTUAL PROPERTY.
    # We are pasting the entire "AI Agent Design Document" here.
    # This turns a generic AI into OUR expert Empathos agent.
    system_prompt = """
    You are an expert qualitative user researcher named 'Empathos'.
    Your prime directive is to demonstrate empathy to understand the user's "why."
    ... (The rest of our entire Design Document goes in here) ...
    """

    headers = {
        "x-api-key": API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "max_tokens": 1024,
        "system": system_prompt, # Our secret sauce!
        "messages": [
            {"role": "user", "content": user_text}
        ]
    }

    # We use httpx to send the request over the internet.
    async with httpx.AsyncClient() as client:
        response = await client.post(
            "https://api.anthropic.com/v1/messages",
            headers=headers,
            json=payload,
            timeout=30.0 # Wait up to 30 seconds for a response
        )

    response_data = response.json()

    # We extract just the text part of the AI's response to send back.
    ai_text = response_data['content'][0]['text']
    return ai_text